In [1]:
import pandas as pd

In [2]:
blast_inputs_mapping = pd.read_csv('./blast_input_mapping.csv',
            names=["subject", "query"]
           )
blast_inputs_mapping.head()

,subject,query
0,b4484.faa,b4484_alleles.fasta


In [3]:
ALLELE_FASTA_PATH = './data/allele_fastas/'

In [4]:
gene_allele_names_d = dict()
for fasta_file in blast_inputs_mapping['query']:
    gene_name = fasta_file.replace("_alleles.fasta", '')
    gene_allele_names_d[gene_name] = set()
    fasta_path = ALLELE_FASTA_PATH + fasta_file
    print(fasta_path)
    with open(fasta_path, 'r') as f:
        for line in f:
            if line.startswith('>'):
                allele_name = line.replace('>','').rstrip("\n")
                gene_allele_names_d[gene_name].add(allele_name)

./data/allele_fastas/b4484_alleles.fasta


In [5]:
import glob

# TODO: should be using blast_inputs_mapping instead of what's in the ref_fastas dir
gene_aa_len_d = dict()
# for ref_fasta_file in glob.glob('./data/ref_fastas/*.faa'):
for ref_fasta_file in blast_inputs_mapping['subject']:
    gene_name = ref_fasta_file.replace('.faa','')
    ref_fasta_file_path = './data/ref_fastas/' + ref_fasta_file
    with open(ref_fasta_file_path, 'r') as file:
        aa_seq = ''
        for line in file:
            if '>' not in line:
                aa_seq += line.rstrip("\n")
        gene_aa_len_d[gene_name] = len(aa_seq)

In [6]:
# String parsing helper functions

# fast find all chars in string: https://stackoverflow.com/questions/52452911/finding-all-positions-of-a-character-in-a-string
def _find_all_idx(string, character):
    idx = string.find(character)
    while idx != -1:
        yield idx
        idx = string.find(character, idx + 1)
        

def _get_char_idxs_in_str(s, c):
    char_idxs = list(_find_all_idx(s, c))
    char_idxs = [x for x in char_idxs]
    return char_idxs
    

CHAR_OF_INTEREST = '-'
assert(_get_char_idxs_in_str('AR-D-G', CHAR_OF_INTEREST)==[2,4])

In [7]:
# BLAST results glossary: https://www.ncbi.nlm.nih.gov/books/NBK62051/
# Description of special characters on the match string: https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=BlastHelp
# The databases alignments are displayed as pairs of matches between query and subject sequence.
# A middle line between the query and subject sequence displays the status of a letter.
# For protein alignments (e.g, BLASTP/BLASTX/TBLASTN), identities present the letter,
# conservative substitutions present a "+", and nothing otherwise. This is the default view.
# https://www.ncbi.nlm.nih.gov/books/NBK62051/: "If the aligned residues have similar physico-chemical properties or have a
# positive score in the governing scoring matrix the substitution is said to be conservative."
# '-' in subject string indicate an insertion

# BLAST subject and query results string are always the same length.


# Won't find differences in the beginning of genes since doesn't compare query vs subject start and end match positions.
# This logic was removed since alleles can have enormous differences on either of their sequences.

# Best to keep the returned data seperate for finding
# consecutive mutations according to AA positions.

BLAST_INDEL_CHAR = '-'
def _get_ins_d(subject_str, query_str, subject_match_start_pos):
    ins_d = dict()
    for idx in _get_char_idxs_in_str(subject_str, BLAST_INDEL_CHAR):
        pos = idx+subject_match_start_pos
        ins_d[pos] = query_str[idx]
    return ins_d

# Won't find differences in the beginning of genes since doesn't compare query vs subject start and end match positions.
def _get_del_d(subject_str, query_str, subject_match_start_pos):
    del_d = dict()
    for idx in _get_char_idxs_in_str(query_str, BLAST_INDEL_CHAR):
        pos = idx+subject_match_start_pos
        del_d[pos] = subject_str[idx]
    return del_d
    

sbjct = 'AR-D-G'  # insertion @ 3 and 5
qry = '-RBDHG'  # delection @ 1
assert(_get_ins_d(sbjct, qry, 1)=={3: 'B', 5: 'H'})
assert(_get_ins_d(sbjct, qry, 2)=={4: 'B', 6: 'H'})  # test effects of subject_match_start_pos

assert(_get_del_d(sbjct, qry, 1)=={1: 'A'})
assert(_get_del_d(sbjct, qry, 2)=={2: 'A'})  # test effects of subject_match_start_pos

In [8]:
ALL_AMINO_ACIDS = {'A','R','N','D','B','C','E','Q','Z','G','H','I','L','K','M','F','P','S','T','W','Y','V'}

def _get_sub_aa_d(subject_str, match_str, query_str, indel_aa_pos_l, subject_match_start_pos):
    seq_chng_d = dict()
    
    for c_i in range(0, len(match_str)):
        if match_str[c_i] not in ALL_AMINO_ACIDS:  # finding BLAST characters indicating mismatch or gap
            pos = subject_match_start_pos+c_i  # subject_match_start_pos+c_i since subject_match_start_pos represents the first charater and the index c_i == 0
            if pos not in indel_aa_pos_l:
                seq_chng_d[pos] = {'s':subject_str[c_i], 'q':query_str[c_i]}
    
    return seq_chng_d


assert(_get_sub_aa_d('ARPGCQ', '+ PG Q', 'MTPGBQ', [], 1)=={1: {'s':'A', 'q':'M'}, 2: {'s':'R', 'q':'T'}, 5: {'s':'C', 'q':'B'}})
assert(_get_sub_aa_d('ARPGCQ', '+ PG Q', 'MTPGBQ', [5], 1)=={1: {'s':'A', 'q':'M'}, 2: {'s':'R', 'q':'T'}})  # testing use of indel_aa_pos_l
assert(_get_sub_aa_d('ARPGCQ', '+ PG Q', 'MTPGBQ', [], 2)=={2: {'s':'A', 'q':'M'}, 3: {'s':'R', 'q':'T'}, 6: {'s':'C', 'q':'B'}})  # test effects of subject_match_start_pos

In [9]:
from itertools import groupby
from operator import itemgetter


def _get_consecutive_pos_l(pos_l):
    cnsc_pos_l = []
    for k, g in groupby(enumerate(pos_l), lambda ix : ix[0] - ix[1]):
        cnsc_pos_l.append(list(map(itemgetter(1), g)))
    return cnsc_pos_l


mut_pos_l = [1, 4,5, 7,8,9, 22]
assert(_get_consecutive_pos_l(mut_pos_l)==[[1], [4, 5], [7, 8, 9], [22]])

In [10]:
import pandas as pd

blast_inputs_mapping['blast output'] = blast_inputs_mapping['query'].apply(lambda s: 'blast_' + s.replace('.fasta', '.xml'))
blast_inputs_mapping.head()

,subject,query,blast output
0,b4484.faa,b4484_alleles.fasta,blast_b4484_alleles.xml


In [11]:
import glob
from Bio.Blast import NCBIXML


gene_var_df = pd.DataFrame()

for blast_output_file in blast_inputs_mapping['blast output']:
    blast_output_file_path = './data/blast_output/' + blast_output_file
    gene = blast_output_file.replace('blast_','').replace('_alleles.xml','')
    
    for record in NCBIXML.parse(open(blast_output_file_path)):
        if len(record.alignments) > 0:
            # Description of available members: https://biopython.org/docs/1.75/api/Bio.Blast.Record.html
            subject_match_start_pos = record.alignments[0].hsps[0].sbjct_start
            blast_subject_str = record.alignments[0].hsps[0].sbjct
            blast_match_str = record.alignments[0].hsps[0].match
            blast_query_str = record.alignments[0].hsps[0].query
            
            # Won't find differences in the beginning of genes since doesn't compare query vs subject start and end match positions.
            ins_d = _get_ins_d(blast_subject_str, blast_query_str, subject_match_start_pos)
            for cnsc_pos_l in _get_consecutive_pos_l(ins_d.keys()):
                seq_chng_str = ''.join([ins_d[p] for p in cnsc_pos_l])
                df = pd.DataFrame.from_dict({
                    'gene': gene,
                    "mutation source": "pangenome variant",
                    "AA range": (min(cnsc_pos_l),max(cnsc_pos_l)),
                    "mutation size": len(cnsc_pos_l),
                    "AA mutation type": "insertion",
                    'AA ref seq': '',
                    "AA seq change": seq_chng_str,
                }, orient='index')  # Can't use columns due to diff parsing of 'AA range' input
                df = df.T
                gene_var_df = pd.concat([df, gene_var_df]) # Accounting for multiples of same allele
            
            # Won't find differences in the beginning of genes since doesn't compare query vs subject start and end match positions.
            del_d = _get_del_d(blast_subject_str, blast_query_str, subject_match_start_pos)
            for cnsc_pos_l in _get_consecutive_pos_l(del_d.keys()):
                seq_chng_str = ''.join([del_d[p] for p in cnsc_pos_l])
                df = pd.DataFrame.from_dict({
                    'gene': gene,
                    "mutation source": "pangenome variant",
                    "AA range": (min(cnsc_pos_l),max(cnsc_pos_l)),
                    "mutation size": len(cnsc_pos_l),
                    "AA mutation type": "deletion",
                    'AA ref seq': seq_chng_str,
                    "AA seq change": '',
                }, orient='index')  # Can't use columns due to diff parsing of 'AA range' input
                df = df.T
                gene_var_df = pd.concat([df, gene_var_df]) # Accounting for multiples of same allele
                
            sub_d = _get_sub_aa_d(
                blast_subject_str,
                blast_match_str,
                blast_query_str,
                list(ins_d.keys()) + list(del_d.keys()),
                subject_match_start_pos)
            
            for cnsc_pos_l in _get_consecutive_pos_l(sub_d.keys()):
                seq_ref_str = ''.join([sub_d[p]['s'] for p in cnsc_pos_l])
                seq_chng_str = ''.join([sub_d[p]['q'] for p in cnsc_pos_l])
                df = pd.DataFrame.from_dict({
                    'gene': gene,
                    "mutation source": "pangenome variant",
                    "AA range": (min(cnsc_pos_l),max(cnsc_pos_l)),
                    "mutation size": len(cnsc_pos_l),
                    "AA mutation type": "substitution",
                    'AA ref seq': seq_ref_str,
                    "AA seq change": seq_chng_str,
                }, orient='index')  # Can't use columns due to diff parsing of 'AA range' input
                df = df.T
                gene_var_df = pd.concat([df, gene_var_df]) # Accounting for multiples of same allele

gene_var_df

,gene,mutation source,AA range,mutation size,AA mutation type,AA ref seq,AA seq change
0,b4484,pangenome variant,"(120, 120)",1,substitution,M,T
0,b4484,pangenome variant,"(105, 105)",1,substitution,E,D
0,b4484,pangenome variant,"(164, 164)",1,substitution,R,H
0,b4484,pangenome variant,"(11, 11)",1,substitution,S,L
0,b4484,pangenome variant,"(147, 147)",1,substitution,T,A
...,...,...,...,...,...,...,...
0,b4484,pangenome variant,"(125, 125)",1,substitution,T,K
0,b4484,pangenome variant,"(153, 153)",1,substitution,S,T
0,b4484,pangenome variant,"(27, 27)",1,substitution,G,V
0,b4484,pangenome variant,"(28, 28)",1,substitution,D,Y
